In [1]:
import numpy as np
import pandas as pd

from pyspark.sql import SQLContext

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pylab as plt

import warnings
warnings.filterwarnings('ignore')
import DataCollector as collector
dc = collector.DataCollector()
import TrainingConfigurations as configurations
t = configurations.TrainingConfigurations()

dc._init_()
data = dc.loadData()
spark = dc.getSparkSession()

In [2]:
import shutil
import os

path = os.path.join('new_message_tracking/data/')
shutil.rmtree(path)
os.mkdir(path)

In [3]:
def print_disk_space(text):
    print(text)
    !df | grep 'overlay'

In [4]:
#%%capture
evals = {}
lgb_estimator = None
counter = 0
n_splits = 10
count = data.count()
each_len = int(count/n_splits)

print('Anzahl Datensätze=' + str(count) + ' each_len=' + str(each_len))

inputColumns = ['CSTARTTIME','CSERVICE','CSENDERENDPOINTID','CSENDERPROTOCOL','CINBOUNDSIZE','year', 'month', 'day', 'hour', 'minute']

lgb_params = {
    'application' : 'binary',
    'num_leaves' : '20'}

while counter < n_splits:
    print_disk_space('Durchlauf Start ' +str(counter))
    df_py = data.limit(each_len)
    data = data.subtract(df_py)
    
    print_disk_space('After substracting data ' +str(counter))
    evals_result = {}

    ###
    df = df_py.toPandas()
    print_disk_space('df_py persisted ' +str(counter))
    df_py.unpersist()
    print_disk_space('df_py unpersisted ' +str(counter))
    xtrain, xother, ytrain, yother = train_test_split(df[inputColumns], df['outcome'], test_size=0.3, shuffle=True)
    del df
    print_disk_space('df deleted ' +str(counter))
    xtest, xvalid, ytest, yvalid = train_test_split(xother, yother, test_size=0.5, shuffle=True)
    
    test_data = pd.concat([xtest,ytest], axis=1)
    train_data = pd.concat([xtrain,ytrain], axis=1)
    valid_data = pd.concat([xvalid,yvalid], axis=1)
    
    test_data.to_csv(t.all_train_path, mode='a')
    train_data.to_csv(t.all_test_path, mode='a')
    valid_data.to_csv(t.all_valid_path, mode='a')
    '''
    #train_data, test_data, valid_data = df.randomSplit([0.8, 0.1, 0.1], seed=1)
    train_data, test_data = df.randomSplit([0.8, 0.2], seed=1)
    
    df.unpersist()

    x_train = test_data[inputColumns]
    y_train = test_data.select('outcome')
    x_test = test_data[inputColumns]
    y_test = test_data.select('outcome')
    #xvalid = test_data[inputColumns]
    #yvalid = test_data.select('outcome')
    
    train_data.unpersist()
    test_data.unpersist()
    
    x_train.write.parquet("new_message_tracking/data/x_train/x_train_" + str(counter) + ".parquet")
    xtrain = x_train.toPandas()
    x_train.unpersist()
    
    y_train.write.parquet("new_message_tracking/data/y_train/y_train_" + str(counter) + ".parquet")
    ytrain = y_train.toPandas()
    y_train.unpersist()
    
    x_test.write.parquet("new_message_tracking/data/x_test/x_test_" + str(counter) + ".parquet")
    xtest = x_test.toPandas()
    x_test.unpersist()
        
    y_test.write.parquet("new_message_tracking/data/y_test/y_test_" + str(counter) + ".parquet")
    ytest = y_test.toPandas()
    y_test.unpersist()
    
    #xvalid = xvalid.toPandas()
    #yvalid = yvalid.toPandas()
    '''
   
    lgb_estimator = lgb.train(lgb_params,
                         keep_training_booster = True,
                         init_model=lgb_estimator,
                         #evals_result = evals_result,
                         callbacks = [lgb.record_evaluation(evals_result)],
                         train_set = lgb.Dataset(xtrain, ytrain),
                         #valid_sets=[lgb.Dataset(xvalid, yvalid), lgb.Dataset(xtrain, ytrain)],
                         #valid_names = ['valid','train'],
                         valid_sets=[lgb.Dataset(xvalid, yvalid), lgb.Dataset(xtrain, ytrain)],
                         valid_names = ['test','train'],
                         num_boost_round=10)

    evals = t.mergeDictionary(evals, evals_result)
    del xtrain, ytrain, xtest, ytest, xvalid, yvalid, test_data, valid_data, train_data
    print_disk_space('variables deleted ' +str(counter))
    print_disk_space('Durchlauf Ende ' +str(counter))
    counter = counter + 1
#    if counter == 2:
#        break

Anzahl Datensätze=233390980 each_len=23339098
Durchlauf Start 0
overlay                209608708 86246872 123361836  42% /
Anzahl Datensätze=210051882
Durchlauf Ende 0
overlay                209608708 141529656  68079052  68% /
Durchlauf Start 1
overlay                209608708 141529656  68079052  68% /
Anzahl Datensätze=186712784
Durchlauf Ende 1
overlay                209608708 195218212  14390496  94% /
Durchlauf Start 2
overlay                209608708 195218212  14390496  94% /


Py4JJavaError: An error occurred while calling o102.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 28 in stage 107.0 failed 1 times, most recent failure: Lost task 28.0 in stage 107.0 (TID 3093) (260c22397ee8 executor driver): java.io.IOException: No space left on device
	at java.base/java.io.FileOutputStream.writeBytes(Native Method)
	at java.base/java.io.FileOutputStream.write(FileOutputStream.java:354)
	at org.apache.spark.storage.TimeTrackingOutputStream.write(TimeTrackingOutputStream.java:59)
	at org.apache.spark.io.MutableCheckedOutputStream.write(MutableCheckedOutputStream.scala:43)
	at java.base/java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:81)
	at java.base/java.io.BufferedOutputStream.write(BufferedOutputStream.java:127)
	at net.jpountz.lz4.LZ4BlockOutputStream.flushBufferedData(LZ4BlockOutputStream.java:223)
	at net.jpountz.lz4.LZ4BlockOutputStream.write(LZ4BlockOutputStream.java:176)
	at java.base/java.io.BufferedOutput

In [ ]:
'''train = spark.read.parquet('new_message_tracking/data/train/')
test = spark.read.parquet('new_message_tracking/data/test/')
valid = spark.read.parquet('new_message_tracking/data/valid/')'''

In [ ]:
x_train = t.get_X_train()
y_train = t.get_y_train()
x_test = t.get_X_test()
y_test = t.get_y_test()
x_valid = t.get_X_valid()
y_valid = t.get_y_valid()

# Model Prediction

In [ ]:
print('test label count: ' + str(t.get_y_test_count()))
print('train label count: ' + str(t.get_y_train_count()))
print('valid label count: ' + str(t.get_y_valid_count()))

In [ ]:
print('test data count: ' + str(t.get_X_test_count()))
print('train data count: ' + str(t.get_X_train_count()))
print('valid data count: ' + str(t.get_X_valid_count()))

In [ ]:
preds = []
preds = lgb_estimator.predict(x_test)

In [ ]:
preds_as_int = []
for pred in preds:
    preds_as_int.append(round(pred))

In [ ]:
len(preds_as_int)

In [ ]:
preds_as_int

In [ ]:
evals

# View Accuracy

In [ ]:
import matplotlib.pyplot as plt
print('Plot metrics during training...')
ax = lgb.plot_metric(evals)
plt.show()

In [ ]:
print('Plot feature importances...')
ax = lgb.plot_importance(lgb_estimator, max_num_features=10)
plt.show()

In [ ]:
unique, counts = np.unique(preds_as_int, return_counts=True)
preds_count = np.asarray((unique, counts))
print(str(preds_count[0][1]) + '   ' + str(preds_count[1][1]))
print(str(preds_count[0][0]) + '   ' + str(preds_count[1][0]))

In [ ]:
y_test.value_counts()

In [ ]:
from sklearn.metrics import mean_squared_error
print('\nThe rmse of prediction is:', mean_squared_error(y_test, preds_as_int) ** 0.5)

In [ ]:
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, preds_as_int)))

##### Confusion-matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, preds_as_int)
print('Confusion matrix\n\n', cm)
print('\nTrue Negatives(TN) = ', cm[0,0])
print('\nTrue Positives(TP) = ', cm[1,1])
print('\nFalse Negatives(FN) = ', cm[0,1])
print('\nFalse Positives(FP) = ', cm[1,0])

In [ ]:
import seaborn as sns
cm_matrix = pd.DataFrame(data=cm, columns=['Actual Negative:0', 'Actual Positive:1'], 
                                 index=['Predict Negative:0', 'Predict Positive:1'])

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

##### Classification Metrices

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds_as_int))

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(y_test, preds_as_int))